# Chinook Digital Music Shop

The [Chinook database](https://github.com/lerocha/chinook-database) contains information about fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, albums, invoices, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Below is a schema diagram for the Chinook database to understand the available columns and the structure of the data:

<img src="https://s3.amazonaws.com/dq-content/189/chinook-schema.svg" width=600 height=400/>

Source: [DataQuest](https://www.dataquest.io/)

In this project, we're going to answer business questions using SQL. Qustions like what type of genre and track are sold in USA, collate data on purchases from different countries, categorize each invoice as either an album purchase or not, and so on..

Before we proceed with our analysis, we need to connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table
